# Problema da Mochila

<div style="text-align:justify">
O problema da mochila (em <i>inglês</i>, Knapsack Problem) é um problema de optimização combinatória. O nome dá-se devido ao modelo de uma situação em que é necessário preencher uma mochila com objetos de diferentes pesos e valores. O objetivo é que se preencha a mochila com o maior valor possível, não ultrapassando o peso máximo.
</div>

## Variáveis

Seja:<br>
- C = Capacidade da mochila
- v<sub>i</sub> = Valor do <i>i-ésimo</i> objeto
- w<sub>i</sub> = Peso do <i>i-ésimo</i> objeto
- x<sub>i</sub> = x<sub>i</sub> &isin; {0, 1} &frasl; x<sub>i</sub> &sub; mochila

## Função Objetivo
<br>
Maximizar o valor dos itens escolhidos.

$$ max \sum_{i=0}^{n} v_i.x_i $$

## Restrição
<br>
O peso total dos itens escolhidos não pode exceder a capacidade da mochila.

$$ \sum_{i=0}^{n} w_i.x_i \leq C $$

## Implementação

<div style="text-align:justify">
Vamos implementar a solução modelada utilizando a Gurobi API. Para isso, vamos criar dados sintéticos que representem o peso e o valor dos objetos a serem adicionados na mochila.
</div>

#### Carregando Dependências

In [1]:
import gurobipy, pandas

#### Dados

In [2]:
C = 1000

df = pandas.read_csv('datasets/knapsack.csv')
weights = df['weights'].values
values = df['values'].values

#### Lista de Itens

In [3]:
items = []
for i in range(len(weights)):
    items.append('item_{}'.format(i))

print(items)

['item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'item_6', 'item_7', 'item_8', 'item_9', 'item_10', 'item_11', 'item_12', 'item_13', 'item_14']


#### Dicionário de Pesos

In [4]:
w = {}
for idx, value in enumerate(weights):
    w[items[idx]] = value

print(w)

{'item_0': 65, 'item_1': 94, 'item_2': 119, 'item_3': 59, 'item_4': 149, 'item_5': 114, 'item_6': 57, 'item_7': 136, 'item_8': 100, 'item_9': 150, 'item_10': 122, 'item_11': 117, 'item_12': 120, 'item_13': 130, 'item_14': 133}


#### Dicionário de Valores

In [5]:
v = {}
for idx, value in enumerate(values):
    v[items[idx]] = value

print(v)

{'item_0': 455, 'item_1': 691, 'item_2': 833, 'item_3': 425, 'item_4': 1064, 'item_5': 758, 'item_6': 419, 'item_7': 914, 'item_8': 651, 'item_9': 966, 'item_10': 828, 'item_11': 827, 'item_12': 857, 'item_13': 837, 'item_14': 894}


#### Modelagem

In [6]:
model = gurobipy.Model()

# decision variables
x = model.addVars(items, vtype=gurobipy.GRB.BINARY)

# objective to optimize
model.setObjective(
    gurobipy.quicksum([v[i] * x[i] for i in items]),
    sense=gurobipy.GRB.MAXIMIZE
)

# constraint 1
c1 = model.addConstr(
    gurobipy.quicksum([w[i] * x[i] for i in items]) <= C
)

model.optimize()

Restricted license - for non-production use only - expires 2022-01-13
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 15 columns and 15 nonzeros
Model fingerprint: 0x397a9a66
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [6e+01, 2e+02]
  Objective range  [4e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective 6210.0000000
Presolve removed 1 rows and 15 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.26 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 6987 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.987000000000e+03, best bound 6.987000000000e+03, gap 0.0000%


#### Itens Selecionados

In [7]:
approved = []
for i in items:
    if round(x[i].X) > 0:
        approved.append(i)

print(approved)

['item_0', 'item_1', 'item_2', 'item_3', 'item_5', 'item_6', 'item_10', 'item_11', 'item_12', 'item_14']


#### Valor Acumulado

In [8]:
print('Valor Objetivo: {}'.format(model.objVal))

Valor Objetivo: 6987.0


#### Capacidade Utilizada

In [9]:
print('Capacidade Utilizada: {}'.format(C - c1.Slack))

Capacidade Utilizada: 1000.0
